In [1]:
# pip install -qU langchain \
#     langchain-community \
#     langchain-together \
#     langchain-core \
#     langchain-google-vertexai \
#     gpt4all \
#     faiss-cpu \
#     faiss-gpu \
#     langgraph \
#     chromadb \
#     sentence-transformers \
#     tavily-python \
#     gradio

In [2]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import nest_asyncio
from langchain_community.vectorstores.chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_together import ChatTogether
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from typing import Literal
from langchain.chains.combine_documents import stuff
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from IPython.core.display import Markdown
import json
import re

from langchain_core.runnables import (
    RunnableParallel,
    RunnableBranch,
    RunnablePassthrough,
)

from langchain_google_vertexai import ChatVertexAI

from langchain_core.messages import HumanMessage, AIMessage
import asyncio
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

# Updated imports from pydantic directly (removing the deprecated langchain_core.pydantic_v1)
from pydantic import BaseModel, Field, validator

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

# Ensure the USER_AGENT environment variable is set (if not already set in .env or system)
# load_dotenv()
# if not os.getenv("USER_AGENT"):
#     os.environ["USER_AGENT"] = "your_user_agent_here"  # You should define your user agent if it's required.


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# import os, getpass
# print("Enter your Tavily API Key")
# TAVILY_API_KEY = getpass.getpass()
# print("Enter your Together API Key")
# TOGETHER_API_KEY = getpass.getpass()

# os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
# os.environ["TOGETHER_API_KEY"] = TOGETHER_API_KEY

In [4]:
urls = [
    "https://www.webmd.com/a-to-z-guides/malaria",
    "https://www.webmd.com/diabetes/type-1-diabetes",
    "https://www.webmd.com/diabetes/type-2-diabetes",
    "https://www.webmd.com/migraines-headaches/migraines-headaches-migraines",
]
loader = WebBaseLoader(urls, bs_get_text_kwargs={"strip": True})
docs = loader.load()
docs

[Document(metadata={'source': 'https://www.webmd.com/a-to-z-guides/malaria', 'title': 'Malaria: Causes, Symptoms, and Treatment', 'description': 'Malaria can be serious and sometimes life-threatening. Learn more about the disease and its new vaccine.', 'language': 'en'}, page_content='Malaria: Causes, Symptoms, and TreatmentSkip to main contentHomeConditionsBackConditionsView AllADD/ADHDAllergiesArthritisAtrial fibrillationBreast CancerCancerCrohn\'s DiseaseDepressionDiabetesDVTEczemaEye HealthHeart DiseaseHIV & AIDSLung DiseaseLupusMental HealthMultiple SclerosisMigrainePain ManagementPsoriasisPsoriatic ArthritisRheumatoid ArthritisSexual ConditionsSkin ProblemsSleep DisordersUlcerative ColitisView AllDrugs & SupplementsBackDrugs & SupplementsDrugsSupplementsPill IdentifierInteraction CheckerWell-BeingBackWell-BeingAging WellBabyBirth ControlChildren\'s HealthDiet & Weight ManagementFitness & ExerciseFood & RecipesHealth & BalanceHealthy BeautyMen\'s HealthParentingPet HealthPregnancy

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
chunks = text_splitter.split_documents(docs)

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
model

c:\Users\Alchemist\Desktop\LangChainProject\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [7]:
embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',show_progress=True, multi_process=True)
embedding_function

HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=True, show_progress=True)

In [8]:
vector_store = FAISS.from_documents(documents=chunks, embedding=embedding_function)

In [9]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023EDCEE47A0>, search_kwargs={'k': 3})

In [ ]:
# from langchain_together import ChatTogether
# from together import Together

# llm = ChatTogether(model='meta-llama/Llama-Vision-Free')
# api_key = os.getenv('TOGETHER_API_KEY')
# client = Together(api_key=api_key)

# for manual setting prompt 
# print(api_key)
# response = client.chat.completions.create(
#     model="meta-llama/Llama-Vision-Free",
#     messages=[{"role": "user", "content": "What are some fun things to do in New York?"}],
# )
# print(response.choices[0].message.content)

# easier and more autmated message return format
from langchain_together import ChatTogether

load_dotenv()

api_key = os.getenv('TOGETHER_API_KEY')

llm = ChatTogether(model='meta-llama/Llama-Vision-Free', api_key=api_key)

# Example usage
response = llm.invoke("What are some fun things to do in New York?")
print(response)



content="The city that never sleeps! New York City is a vibrant and diverse destination that offers endless options for entertainment, culture, and adventure. Here are some fun things to do in New York:\n\n**Iconic Landmarks and Attractions:**\n\n1. **Statue of Liberty and Ellis Island**: Take a ferry to Liberty Island to see the iconic statue up close and visit the Ellis Island Immigration Museum.\n2. **Central Park**: A sprawling green oasis in the middle of Manhattan, perfect for walking, jogging, or picnicking.\n3. **Empire State Building**: Enjoy panoramic views of the city from the observation deck on the 86th floor.\n4. **Times Square**: Experience the bright lights, giant billboards, and lively atmosphere of this iconic square.\n5. **9/11 Memorial & Museum**: A poignant tribute to the victims of the 9/11 attacks.\n\n**Arts and Culture:**\n\n1. **Museum of Modern Art (MoMA)**: One of the world's premier modern art museums, with a collection of over 200,000 works.\n2. **Metropoli